 **Homework 2 Phoneme Classification**

* Slides: https://docs.google.com/presentation/d/1v6HkBWiJb8WNDcJ9_-2kwVstxUWml87b9CnA16Gdoio/edit?usp=sharing
* Kaggle: https://www.kaggle.com/c/ml2022spring-hw2
* Video: TBA


In [2]:
!nvidia-smi

'nvidia-smi' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


## Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`
- `libriphone/train_labels`
- `libriphone/test_split.txt`
- `libriphone/feat/train/*.pt`: training feature<br>
- `libriphone/feat/test/*.pt`:  testing feature<br>

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2022spring-hw2/data) and upload it to the workspace**


### Download train/test metadata

!pip install --upgrade gdown

# Main link
!gdown --id '1o6Ag-G3qItSmYhTheX6DYiuyNzWyHyTc' --output libriphone.zip

# Backup link 1
# !gdown --id '1R1uQYi4QpX0tBfUWt2mbZcncdBsJkxeW' --output libriphone.zip

# Bqckup link 2
# !wget -O libriphone.zip "https://www.dropbox.com/s/wqww8c5dbrl2ka9/libriphone.zip?dl=1"

!unzip -q libriphone.zip
!ls libriphone

### Preparing Data

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [2]:
import torch

x = torch.randn(1, 3, 224, 224)
y = x.repeat(3, 1, 1, 1)
print(x.shape)
print(y.shape)

torch.Size([1, 3, 224, 224])
torch.Size([3, 3, 224, 224])


In [4]:
# torch.empty:用来返回一个没有初始化的tensor
torch.empty(2,3)
'''
tensor([[1.1692e-19, 1.5637e-01, 5.0783e+31],
        [4.2964e+24, 2.6908e+20, 2.7490e+20]])
'''


'\ntensor([[1.1692e-19, 1.5637e-01, 5.0783e+31],\n        [4.2964e+24, 2.6908e+20, 2.7490e+20]])\n'

In [5]:
# pytorch中view函数的作用为重构张量的维度，相当于numpy中resize（）的功能
import torch

a=torch.Tensor([[[1,2,3],[4,5,6]]])
b=torch.Tensor([1,2,3,4,5,6])

print(a.view(1,6))
print(b.view(1,6))

tensor([[1., 2., 3., 4., 5., 6.]])
tensor([[1., 2., 3., 4., 5., 6.]])


In [3]:
a = torch.randn(936,39)
print(a)
print(a.shape)
print(a[0])
print(a[0].shape)

tensor([[-0.3136, -0.2237,  0.9403,  ...,  0.4682,  1.0104, -0.9377],
        [-1.2905, -0.8121,  0.1026,  ..., -2.0334,  1.1495, -1.4538],
        [-0.8909, -1.0733,  0.4083,  ..., -0.4212,  0.3736, -0.9635],
        ...,
        [-0.1460,  0.1726,  0.8799,  ...,  1.2711, -0.2048,  0.7808],
        [-1.5607,  0.8653,  1.7588,  ...,  0.8852, -0.7899,  1.7406],
        [ 0.9347, -1.9260,  1.1858,  ..., -0.5402, -0.1968, -0.3785]])
torch.Size([936, 39])
tensor([-0.3136, -0.2237,  0.9403, -1.3153, -0.5445,  0.1701,  0.0828, -0.8443,
         0.5517,  0.9706, -0.0939, -0.4608,  1.7830,  1.2224,  0.3265,  1.2890,
         0.3121,  0.7995, -0.7957,  2.0888,  0.2653,  0.1460, -0.2003, -1.0761,
        -1.5568, -0.7701,  1.0760,  0.2599,  0.8407,  1.9645,  0.8481, -0.0097,
         0.2377, -1.1161,  0.0988,  0.9317,  0.4682,  1.0104, -0.9377])
torch.Size([39])


In [22]:
x = torch.randn(8,4,3) # ??????????

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1) 
        right = x[:n]

    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

mid = 3
for r_idx in range(1, mid+1):
    x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx) # x[mid + r_idx, :]：抽取 x 中的第 mid + r_idx 个数组元素
    x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)


In [6]:
import os
import random
import pandas as pd
import torch
from tqdm import tqdm
from torchsummary import summary

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]

    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    # pytorch中view函数的作用为重构张量的维度，相当于numpy中resize（）的功能
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, train_val_seed=1337):
    # 查看PPT，可以看到 Label: 41 classes, each class represents a phoneme
    class_num = 41 # NOTE: pre-computed, should not need change
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}
    if mode != 'test':
      phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

      for line in phone_file:
          line = line.strip('\n').split(' ')
          label_dict[line[0]] = [int(p) for p in line[1:]]
          
    # 划分 训练集 和 验证集
    if split == 'train' or split == 'val':
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        # 使得随机数据可预测，即只要seed的值一样，后续生成的随机数都一样。这样既可以让结果得以复现。
        # 参考文档：https://blog.csdn.net/qq_38410428/article/details/99655943
        random.seed(train_val_seed)

        # random.shuffle()用于将一个列表中的元素打乱顺序，值得注意的是使用这个方法不会生成新的列表，只是将原列表的次序打乱。
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:] 
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')
    
    # 得到每一个音频代号
    usage_list = [line.strip('\n') for line in usage_list]
    # [Dataset] - # phone classes: 41, number of utterances for val: 858 , 打印 数据集 的信息，包括类别和长度
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    # torch.empty：返回一个没有初始化的tensor
    # 一種流行的音頻特徵提取方法是Mel-frequency cepstral coefficients（MFCC），它有39個特徵。
    X = torch.empty(max_len, 39 * concat_nframes) # 一个 frame 有 39 个dim，将多个 frames 拼接起来，就得到 39 * concat_nframes 个 dim
    if mode != 'test':
        #  torch.long():向下取整
        y = torch.empty(max_len, dtype=torch.long)

    # 将音频数据读取出来 X为特征 y为label
    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode != 'test':
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode != 'test':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode != 'test':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode != 'test':
      print(y.shape)
      return X, y
    else:
      return X


```python
def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]

    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)
```


打印 x[0] 和 left：（示例，只打印了前3），其 shape 分别是：x[0] torch.Size([39])，left torch.Size([1, 39])...

![示例，只打印了前3](1.png)

如果将代码改为如下：

```python
        left = x[0].repeat(-n, 1) # shape：torch.size([1,39]), torch.size([2,39]), torch.size([3,39])...
        left2 = x[0].repeat(1, -n) # 如果是这样, 后续的 shape 会变成 torch.size([1,39]), torch.size([1,78]), torch.size([1,117])......
        print('left.shape',left.shape)
        print('left2.shape',left2.shape)
```
打印结果如下：

![示例，只打印了前3](2.png)

right 打印入下:

![](3.png)

## Define Dataset

In [7]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class LibriDataset(Dataset):
    """
        LibriDataset 继承 Dataset, 重载了__init__(), __getitem__(), __len__()
        实现将一组Tensor数据对封装成Tensor数据集
        能够通过index得到数据集的数据，能够通过len，得到数据集大小
    """

    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        print('self.label:')
        print(self.label)
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


## Define Model

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.block(x)
        return x


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()

        self.fc = nn.Sequential(
            
            # 第一个 BasicBlock，将输入维度转换为隐藏层维度
            BasicBlock(input_dim, hidden_dim),

            # 参考文档：https://blog.csdn.net/xty123abc/article/details/135924236（分类神经网络与drop out）

            # 根据 hidden_layers 数量添加多个 BasicBlock
            # 对 *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)]的一个补充解释:
            #  "*"代表解压列表，例如A=[a,b,c]，那么f(*A)=f(a,b,c)在这里的具体意义是“便于控制隐藏层数量”
            # 而其中的"_"代表不希望在循环中使用变量，这是一种通用的惯例，表明循环的目的不是对每个元素进行操作，而是为了重复某个操作特定次数。
            # 如果hidden_layers=3，这里的等价含义就是BasicBlock(hidden_dim, hidden_dim)，BasicBlock(hidden_dim, hidden_dim)，BasicBlock(hidden_dim, hidden_dim)，——连续出现三次
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x
    
# 打印模型的每一层

## Hyper-parameters

In [9]:
# data prarameters
# 具体拼接多少个 frame，如何确定。。。
concat_nframes = 1              # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.8               # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 0                        # random seed
batch_size = 512                # batch size
num_epoch = 5                   # the number of training epoch
learning_rate = 0.0001          # learning rate
model_path = './model.ckpt'     # the path where the checkpoint will be saved

# model parameters
input_dim = 39 * concat_nframes # the input dim of the model, you should not change the value
hidden_layers = 1               # the number of hidden layers
hidden_dim = 256                # the hidden dim

## Prepare dataset and model

In [10]:
import gc

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

print('train_set === ', train_set)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for train: 3428


3428it [00:02, 1386.50it/s]


[INFO] train set
torch.Size([2116368, 39])
torch.Size([2116368])
[Dataset] - # phone classes: 41, number of utterances for val: 858


858it [00:00, 1378.68it/s]

[INFO] val set
torch.Size([527790, 39])
torch.Size([527790])
train_set ===  <__main__.LibriDataset object at 0x0000020405CB5410>


In [11]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

DEVICE: cpu


In [12]:
import numpy as np

#fix seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [13]:
# fix random seed
same_seeds(seed)

# create model, define a loss function, and optimizer
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)

criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


print(model)
# summary(model, input_size=(input_dim, hidden_layers, hidden_dim)) # 如何查看高度和宽度
summary(model, input_size=(input_dim,))

## Training

In [ ]:
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
 
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad() 
        outputs = model(features) 
        
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    
    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, labels = batch
                features = features.to(device)
                labels = labels.to(device)
                outputs = model(features)
                
                loss = criterion(outputs, labels)
                # 测试模式下，loss、优化都不需要更新
                # loss.backward() 
                # optimizer.step()  
                
                _, val_pred = torch.max(outputs, 1) 
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')

# 由以下结果看出模型欠拟合，需要提高模型复杂度


100%|██████████| 1031/1031 [00:05<00:00, 195.81it/s]


[001/005] Train Acc: 0.421913 Loss: 2.086596 | Val Acc: 0.440635 loss: 1.971876
saving model with acc 0.441


100%|██████████| 1031/1031 [00:05<00:00, 195.80it/s]


[002/005] Train Acc: 0.449041 Loss: 1.934451 | Val Acc: 0.449760 loss: 1.926819
saving model with acc 0.450


100%|██████████| 1031/1031 [00:05<00:00, 197.67it/s]


[003/005] Train Acc: 0.455017 Loss: 1.904329 | Val Acc: 0.453819 loss: 1.908851
saving model with acc 0.454


100%|██████████| 1031/1031 [00:05<00:00, 194.13it/s]


[004/005] Train Acc: 0.458421 Loss: 1.887973 | Val Acc: 0.456102 loss: 1.896483
saving model with acc 0.456


100%|██████████| 1031/1031 [00:06<00:00, 152.09it/s]

[005/005] Train Acc: 0.460745 Loss: 1.876421 | Val Acc: 0.457870 loss: 1.889744
saving model with acc 0.458


In [ ]:
del train_loader, val_loader
gc.collect() # Python内置的垃圾回收函数，用于手动触发垃圾回收器。

7

## Testing
Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# load data
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 1078


1078it [00:00, 2617.84it/s]


[INFO] test set
torch.Size([646268, 39])


In [ ]:
# load model
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
#model.load_state_dict()函数把加载的权重复制到模型的权重中去
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [ ]:
test_acc = 0.0
test_lengths = 0
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 1263/1263 [00:05<00:00, 220.99it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))